In [ ]:
from datasets import load_dataset,Dataset
import torch
import matplotlib.pyplot as plt
import einx
import numpy as np

In [ ]:
class COCOLoader:

	def __init__(self,
			  mscoco_path):
		
		self.ds = load_dataset("Alljoined/05_125")
		self.mscoco = torch.load(mscoco_path)

		uniques = np.unique(self.ds["train"]["73k_id"])

		for i in uniques:
			assert i in self.mscoco["idx"]

		self.idx_map = {}

		for idx,i in enumerate(self.mscoco["idx"]):
			self.idx_map[i] = idx

	def to_saved_idx(self,indices,map):
		return np.vectorize(map.get)(indices)
	
	def get_batch(self,
			   indices,
			   split="train",
			   label="embeds"):
		
		eeg = self.ds[split][indices]["EEG"]
		subset_idx = self.ds[split][indices]["73k_id"]
		img_idx = self.to_saved_idx(subset_idx,self.idx_map)
		return eeg,self.mscoco[label][img_idx]

In [ ]:
def prep_batch(x,y):
	x = torch.Tensor(x)
	y = einx.rearrange("b 1 d -> b d",y)
	return x,y

In [ ]:
loader = COCOLoader("data/alljoined1/nsd_ms_coco.pt")

In [ ]:
x,y = prep_batch(*loader.get_batch([10,278,28,290,10]))

In [ ]:
x.shape

In [ ]:
y.shape